In [1]:
#!/usr/bin/env python3.7  
# -*- coding: utf-8 -*- 
#----------------------------------------------------------------------------
# Created By  : name_of_the_creator Hugo Andres Dorado   
# Created Date: 17/02/2022
# version ='1.0'

from msilib import sequence
import pandas as pd
import numpy as np
import random as rd
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
from joblib import dump, load
import csv
import re
from Optimization_funs import *

In [2]:

# Read dataset

dataset = pd.read_csv("datasetcordoba.csv",index_col=0)

del dataset["year_sems"]

dataset.columns

X = dataset.copy()

del X["Yield"]

Y = dataset["Yield"]

In [4]:
# Extract ranges

clasVars = pd.read_csv("header_dataset.csv")

In [5]:
# Subset of variables

managment_vars = clasVars[clasVars.Type=="M"]["Variable"].reset_index()["Variable"]
scales_managment_vars = clasVars[clasVars.Type=="M"]["Scale"].reset_index()["Scale"]

other_variables = clasVars[clasVars.Type!="M" ]
other_variables = other_variables[other_variables.Type != "O"]["Variable"].reset_index()["Variable"]

In [6]:
# Split dataset

mat_M = dataset[managment_vars]
mat_O = dataset[other_variables]

In [8]:
# Validation

ds_ranges = AllRangGen(mat_M,scales_managment_vars,managment_vars)


[[84848.48485, 42424.24242],
 [5.0, 1.0],
 [87500.0, 13500.0],
 [3.0, 0.0],
 [5.0, 0.0],
 [12.0, 0.0],
 [0.276, 0.0],
 [0.0475, 0.0],
 [0.18, 0.0],
 [4.0, 1.0],
 ['Manual', 'Mecanizado'],
 ['NO', 'SI'],
 ['ADV 9293 (Syngenta)',
  'ADV 9339 (Syngenta)',
  'Cerato (Syngenta)',
  'DK7088',
  'DK 234',
  'DK 234 YGRR',
  'DK 1040',
  'DK 1596',
  'FNC 114',
  'FNC 3056',
  'ICA V 156',
  'ICA V 305',
  'Impacto (Syngenta)',
  'NK254 (Syngenta)',
  'Otro',
  'P3966 (Pioneer)',
  'P4082 (Pioneer)',
  'PAC 105',
  'PIONEER 30F32',
  'PIONEER 30F32HW',
  'PIONEER 30F35',
  'PIONEER 30F35 H',
  'PIONEER 30F35 HRR',
  'Sinko (Syngenta)',
  'Status (Syngenta)'],
 ['Manual', 'Mecanizada'],
 ['Hibrido', 'OGM']]

In [9]:
# Persistence

ct = load('pipe_line.joblib')

rf = load('rf_cordoba.joblib') 

In [10]:
gen = dataset.iloc[[15]]

gen

,Sowing_Seeds_Number,Seeds_Per_Site,Plant_Density_20_days,Chemical_Treat_Disease,Chemical_Treat_Weeds,Chemical_Treat_Pests,Total_N,Total_P,Total_K,Number_Chemical_Ferti,...,Cultivar,Former_Crop,Field_Drainage,Harvest_Method,Cultivar_Type,Soil_Structure,Runoff,Soil_Texture,Organic_Matter_Content,Yield
ID,,,,,,,,,,,,,,,,,,,,,
16,75000.0,1.0,68000.0,0.0,2.0,1.0,0.092,0.0,0.0,2.0,...,FNC 114,Maiz,NO,Manual,Hibrido,MASIVA,NINGUNO,FrL,MEDIA,4.0


In [11]:
solution = bestGlobHS(fv=gen[other_variables],hms=5,hmcr=0.85,par=0.3,maxNumInp=100,fitnessfun=fitnessfun,
namesds=managment_vars,fixedVars=other_variables,model_train=rf,ranges=ds_ranges,scales=scales_managment_vars,
transf_fun=ct)

In [12]:
solution[1]

Variable,Sowing_Seeds_Number,Seeds_Per_Site,Plant_Density_20_days,Chemical_Treat_Disease,Chemical_Treat_Weeds,Chemical_Treat_Pests,Total_N,Total_P,Total_K,Number_Chemical_Ferti,Sowing_Method,Seeds_Treatment,Cultivar,Harvest_Method,Cultivar_Type,Performance
0,58715.181461,1.496693,81929.186195,2,5,12,0.130543,0.034203,0.110802,3,Mecanizado,NO,PIONEER 30F35,Mecanizada,OGM,5.11286


In [13]:
solution[0]

2     4.529316
3     4.675774
1     4.751210
4     4.758650
0     4.893706
        ...   
95    5.112860
96    5.112860
97    5.112860
98    5.112860
99    5.112860
Name: Performance, Length: 100, dtype: float64